取得 Account ID 驗證 AWS 本機設置正確完成

In [1]:
import boto3

# 載入 .env 檔案中的環境變數
from dotenv import load_dotenv
import os
load_dotenv()

# 建立 STS 客戶端
sts_client = boto3.client("sts")

# 呼叫 get_caller_identity 以取得當前 AWS 帳號資訊
response = sts_client.get_caller_identity()
account_id = response["Account"]

print(f"AWS Account ID: {account_id}")

AWS Account ID: 753251772626


遍歷所有 S3 bucket 中的資源並顯示樹狀結構

In [2]:
import boto3

# 建立 S3 資源對象
s3 = boto3.resource('s3')

def list_s3_bucket_structure(bucket_name):
    bucket = s3.Bucket(bucket_name)
    print(f"Bucket: {bucket_name}")
    
    has_objects = False
    for obj in bucket.objects.all():
        print(" └── ", obj.key)
        has_objects = True

    if not has_objects:
        print(" └── <此 bucket 為空>")

# 取得所有 bucket 名稱並顯示每個 bucket 的樹狀結構
s3_client = boto3.client("s3")
buckets = s3_client.list_buckets()["Buckets"]

if buckets:
    for bucket in buckets:
        list_s3_bucket_structure(bucket["Name"])
else:
    print("當前為空的：沒有任何 S3 bucket")

Bucket: c133864a3391500l8216998t1w75325177262-lambdabucket-gbht6yptzc87
 └──  s3cleaner_function.zip
Bucket: c133864a3391500l8216998t1w753251772626-labbucket-m0jhfwhqvfik
 └──  20_newsgroups.tar.gz


檢查當前使用者在 S3 和 Comprehend 的權限

In [3]:
import boto3
from botocore.exceptions import ClientError

# 檢查 S3 權限
def check_s3_permission():
    s3_client = boto3.client("s3")
    try:
        # 嘗試列出所有 bucket，檢查是否有 S3 權限
        response = s3_client.list_buckets()
        print("S3 權限檢查成功：有權限訪問 S3 服務。")
    except ClientError as e:
        if e.response['Error']['Code'] == 'AccessDenied':
            print("S3 權限不足：無法訪問 S3。錯誤訊息:", e)
        else:
            print("其他 S3 錯誤:", e)

# 檢查 Comprehend 權限
def check_comprehend_permission():
    comprehend_client = boto3.client("comprehend")
    sample_text = "This is a sample text for checking permissions."

    try:
        # 嘗試提取關鍵詞組，檢查是否有 Comprehend 權限
        comprehend_client.detect_key_phrases(Text=sample_text, LanguageCode="en")
        print("Comprehend 權限檢查成功：有權限使用 Comprehend 服務。")
    except ClientError as e:
        if e.response['Error']['Code'] == 'AccessDeniedException':
            print("Comprehend 權限不足：無法使用 Comprehend。錯誤訊息:", e)
        else:
            print("其他 Comprehend 錯誤:", e)

# 執行權限檢查
check_s3_permission()
check_comprehend_permission()


S3 權限檢查成功：有權限訪問 S3 服務。
Comprehend 權限不足：無法使用 Comprehend。錯誤訊息: An error occurred (AccessDeniedException) when calling the DetectKeyPhrases operation: User: arn:aws:sts::753251772626:assumed-role/voclabs/user2069478=_________ is not authorized to perform: comprehend:DetectKeyPhrases because no identity-based policy allows the comprehend:DetectKeyPhrases action


In [4]:
import boto3
from botocore.exceptions import ClientError

# 設定 S3 和 Comprehend 客戶端
s3_client = boto3.client("s3")
comprehend_client = boto3.client("comprehend")

# S3 Bucket 和 IAM Role ARN
bucket = 'c133864a3391500l8216998t1w753251772626-labbucket-m0jhfwhqvfik'
data_access_role_arn = 'arn:aws:iam::753251772626:role/service-role/c133864a3391500l8216998t1w-ComprehendDataAccessRole-BwjcVJXuvgaz'

# 權限檢查：S3 存取權限
def check_s3_access(bucket_name):
    try:
        # 嘗試列出 S3 Bucket 內容以確認權限
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if "Contents" in response:
            print(f"S3 Bucket '{bucket_name}' 存取成功，包含物件數量:", len(response["Contents"]))
        else:
            print(f"S3 Bucket '{bucket_name}' 內無任何物件。")
    except ClientError as e:
        print(f"S3 存取失敗。錯誤訊息: {e}")

# 權限檢查：Comprehend 存取權限
def check_comprehend_access(text_sample, role_arn):
    try:
        # 測試 Comprehend 的 DetectKeyPhrases 權限
        response = comprehend_client.detect_key_phrases(
            Text=text_sample,
            LanguageCode="en"
        )
        print("Comprehend 的 DetectKeyPhrases 操作成功。")
    except ClientError as e:
        print("Comprehend 的 DetectKeyPhrases 操作失敗。錯誤訊息:", e)

    try:
        # 測試 Comprehend 的 DetectEntities 權限
        response = comprehend_client.detect_entities(
            Text=text_sample,
            LanguageCode="en"
        )
        print("Comprehend 的 DetectEntities 操作成功。")
    except ClientError as e:
        print("Comprehend 的 DetectEntities 操作失敗。錯誤訊息:", e)

# 測試樣本文本
text_sample = "This is a sample text for checking permissions in Comprehend."

# 執行 S3 和 Comprehend 的權限檢查
print("正在檢查 S3 權限...")
check_s3_access(bucket)

print("\n正在檢查 Comprehend 權限...")
check_comprehend_access(text_sample, data_access_role_arn)


正在檢查 S3 權限...
S3 Bucket 'c133864a3391500l8216998t1w753251772626-labbucket-m0jhfwhqvfik' 存取成功，包含物件數量: 1

正在檢查 Comprehend 權限...
Comprehend 的 DetectKeyPhrases 操作失敗。錯誤訊息: An error occurred (AccessDeniedException) when calling the DetectKeyPhrases operation: User: arn:aws:sts::753251772626:assumed-role/voclabs/user2069478=_________ is not authorized to perform: comprehend:DetectKeyPhrases because no identity-based policy allows the comprehend:DetectKeyPhrases action
Comprehend 的 DetectEntities 操作失敗。錯誤訊息: An error occurred (AccessDeniedException) when calling the DetectEntities operation: User: arn:aws:sts::753251772626:assumed-role/voclabs/user2069478=_________ is not authorized to perform: comprehend:DetectEntities because no identity-based policy allows the comprehend:DetectEntities action
